In [9]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# 1) Traer data personal activo 
# ========================================
users = models.execute_kw(
    db, uid, password,
    "hr.employee", "search_read",
    [[ ["active", "=", True]]],
    {"fields": ["ubicacion_trabajo","department_id","display_name","work_email","job_id"]}
)
df = pd.DataFrame(users)

df["department_name"] = df["department_id"].apply(lambda x: x[1] if isinstance(x, list) else None)

# Separar la ruta por slash " / "
df[["empresa", "area", "cargo"]] = df["department_name"].str.split("/", expand=True)

# Limpiar espacios
df["area"] = df["area"].str.strip()
df["cargo"] = df["cargo"].str.strip()


In [10]:
#df = df.drop(columns=["empresa","department_name","department_id"])

In [11]:
df[df["display_name"].str.contains("ADRIAN")]

,display_name,job_id,ubicacion_trabajo,work_email,id,department_id,department_name,empresa,area,cargo
87,MIRBELIS ADRIANA MARIN ALBARRAN,"[18, SERVICIOS GENERALES]","[563, [25214] Cundinamarca / Cota]",False,729,"[4, DONSSON / GERENCIA GENERAL / ADMINISTRACION]",DONSSON / GERENCIA GENERAL / ADMINISTRACION,DONSSON,GERENCIA GENERAL,ADMINISTRACION


In [12]:
df_def = df[df["cargo"]=="COMERCIAL"]

In [13]:
df_def.sample(5)

,display_name,job_id,ubicacion_trabajo,work_email,id,department_id,department_name,empresa,area,cargo
39,EVERTH DE JESUS QUICENO JIMENEZ,"[4, VENDEDOR APRENDIZ ]","[143, [05360] Antioquia / Itagüí]",False,679,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL
51,JAIME ANDRES DUARTE MORA,"[9, VENDEDOR EXTERNO]","[1130, [76892] Valle del Cauca / Yumbo]",jaime.duarte@donsson.com,736,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL
105,WILSON ALEXANDER PELAEZ PEREZ,"[9, VENDEDOR EXTERNO]","[85, [05001] Antioquia / Medellín]",wilson.pelaez@donsson.com,513,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL
99,SERGIO DUBAN PACHON MORA,"[36, coordinador despachos y facturación]","[563, [25214] Cundinamarca / Cota]",sergio.pachon@donsson.com,370,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL
62,JUAN DAVID OCHOA BELTRÁN,"[92, INGENIERO DE VENTAS]","[563, [25214] Cundinamarca / Cota]",juandavid.ochoa@donsson.com,725,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL


In [14]:
import re

# Extraer el texto (posición 1 de la lista)
df_def["ubi"] = df_def["ubicacion_trabajo"].apply(
    lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None
)

# Quitar los corchetes con número: [25214]
df_def["ubi"] = df_def["ubi"].apply(
    lambda x: re.sub(r"\[\d+\]\s*", "", x) if isinstance(x, str) else None
)

# Separar asegurando siempre 2 columnas
df_def[["departamento", "ciudad"]] = (
    df_def["ubi"]
    .str.split("/", n=1, expand=True)
    .reindex(columns=[0, 1])  # <-- garantiza que existan 2 columnas
)

df_def = df_def.drop(columns=["ubicacion_trabajo","ubi"])



/tmp/ipykernel_50108/3679110601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_def["ubi"] = df_def["ubicacion_trabajo"].apply(
/tmp/ipykernel_50108/3679110601.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_def["ubi"] = df_def["ubi"].apply(
/tmp/ipykernel_50108/3679110601.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [15]:
df_def.sample(10)

,display_name,job_id,work_email,id,department_id,department_name,empresa,area,cargo,departamento,ciudad
52,JAIME SEBASTIAN RODRIGUEZ RAMOS,"[55, VENDEDOR MOSTRADOR JUNIOR]",sebastian.rodriguez@donsson.com,522,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL,Cundinamarca,Cota
17,CLAUDIA XIMENA ORTIZ CAMACHO,"[55, VENDEDOR MOSTRADOR JUNIOR]",clauximenaortizcamacho@gmail.com,708,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL,Valle del Cauca,Yumbo
50,JAIME ANDRES CABALLERO CABALLERO,"[51, VENDEDOR MOSTRADOR SENIOR]",ventasandres@donsson.com,268,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL,Cundinamarca,Cota
3,ANDRES EMILIO CORRALES CORBY,"[9, VENDEDOR EXTERNO]",andres.corrales@donsson.com,472,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL,Cundinamarca,Cota
97,SANDRA PATRICIA RAMIREZ CORREDOR,"[4, VENDEDOR APRENDIZ ]",False,551,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL,Santander,Bucaramanga
7,AURELYSMAR REA SILVA,"[8, APRENDIZ SENA ]",False,698,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL,Bogotá,"Bogotá, D.C."
5,ANDRES FELIPE ORDOÑEZ GREGORY,"[54, DIRECTOR DE SEDE]",False,297,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL,Cundinamarca,Cota
108,YANID AIMEETH RODRIGUEZ JOTA,"[95, AUXILIAR DE FACTURACIÓN Y DESPACHOS]",yanidjota15@hotmail.com,501,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL,Cundinamarca,Cota
66,JUAN JOSÉ OSPINA CARDONA,"[92, INGENIERO DE VENTAS]",juanjose.ospina@donsson.com,728,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL,Antioquia,Medellín
74,LEONARDO MORA CUBILLOS,"[54, DIRECTOR DE SEDE]",ventasleonardo@donsson.com,293,"[5, DONSSON / GERENCIA GENERAL / COMERCIAL]",DONSSON / GERENCIA GENERAL / COMERCIAL,DONSSON,GERENCIA GENERAL,COMERCIAL,Atlántico,Barranquilla


In [16]:
df_def.to_excel("vendedores.xlsx")